In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
#full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
#choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
#frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
#choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
#frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
#choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
#frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
#choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
#frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
#choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
#frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
#choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
#frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


In [11]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
#choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
#frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


both MDM and RUM represent the data with approximately 99.8% confidence interval,  we set both z = 3.090232306167813 for both RUM and MDM, which corresponding to  99.8% confidence interval

In [12]:
confidence_level = 0.998
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)

stardard_error_20 = np.zeros(frequency_collection_20.shape)
for i in range(stardard_error_20.shape[0]):
    for j in range(stardard_error_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            stardard_error_20[i][j] = np.sqrt((1-choice_collection_20[i][j])/frequency_collection_20[i][j])
            
lb_20 = np.zeros(frequency_collection_20.shape)
ub_20 = np.zeros(frequency_collection_20.shape)
for i in range(lb_20.shape[0]):
    for j in range(lb_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            lb_20[i][j] = choice_collection_20[i][j] * (1-target_z_score*stardard_error_20[i][j]) 
            ub_20[i][j] = choice_collection_20[i][j] * (1+target_z_score*stardard_error_20[i][j])

stardard_error_30 = np.zeros(frequency_collection_30.shape)
for i in range(stardard_error_30.shape[0]):
    for j in range(stardard_error_30.shape[1]):
        if frequency_collection_30[i][j]!=0:
            stardard_error_30[i][j] = np.sqrt((1-choice_collection_30[i][j])/frequency_collection_30[i][j])
            
lb_30 = np.zeros(frequency_collection_30.shape)
ub_30 = np.zeros(frequency_collection_30.shape)
for i in range(lb_30.shape[0]):
    for j in range(lb_30.shape[1]):
        if frequency_collection_30[i][j]!=0:
            lb_30[i][j] = choice_collection_30[i][j] * (1-target_z_score*stardard_error_30[i][j]) 
            ub_30[i][j] = choice_collection_30[i][j] * (1+target_z_score*stardard_error_30[i][j])
            
stardard_error_40 = np.zeros(frequency_collection_40.shape)
for i in range(stardard_error_40.shape[0]):
    for j in range(stardard_error_40.shape[1]):
        if frequency_collection_40[i][j]!=0:
            stardard_error_40[i][j] = np.sqrt((1-choice_collection_40[i][j])/frequency_collection_40[i][j])
            
lb_40 = np.zeros(frequency_collection_40.shape)
ub_40 = np.zeros(frequency_collection_40.shape)
for i in range(lb_40.shape[0]):
    for j in range(lb_40.shape[1]):
        if frequency_collection_40[i][j]!=0:
            lb_40[i][j] = choice_collection_40[i][j] * (1-target_z_score*stardard_error_40[i][j]) 
            ub_40[i][j] = choice_collection_40[i][j] * (1+target_z_score*stardard_error_40[i][j])

stardard_error_50 = np.zeros(frequency_collection_50.shape)
for i in range(stardard_error_50.shape[0]):
    for j in range(stardard_error_50.shape[1]):
        if frequency_collection_50[i][j]!=0:
            stardard_error_50[i][j] = np.sqrt((1-choice_collection_50[i][j])/frequency_collection_50[i][j])
            
lb_50 = np.zeros(frequency_collection_50.shape)
ub_50 = np.zeros(frequency_collection_50.shape)
for i in range(lb_50.shape[0]):
    for j in range(lb_50.shape[1]):
        if frequency_collection_50[i][j]!=0:
            lb_50[i][j] = choice_collection_50[i][j] * (1-target_z_score*stardard_error_50[i][j]) 
            ub_50[i][j] = choice_collection_50[i][j] * (1+target_z_score*stardard_error_50[i][j])
            
stardard_error_60 = np.zeros(frequency_collection_60.shape)
for i in range(stardard_error_60.shape[0]):
    for j in range(stardard_error_60.shape[1]):
        if frequency_collection_60[i][j]!=0:
            stardard_error_60[i][j] = np.sqrt((1-choice_collection_60[i][j])/frequency_collection_60[i][j])
            
lb_60 = np.zeros(frequency_collection_60.shape)
ub_60 = np.zeros(frequency_collection_60.shape)
for i in range(lb_60.shape[0]):
    for j in range(lb_60.shape[1]):
        if frequency_collection_60[i][j]!=0:
            lb_60[i][j] = choice_collection_60[i][j] * (1-target_z_score*stardard_error_60[i][j]) 
            ub_60[i][j] = choice_collection_60[i][j] * (1+target_z_score*stardard_error_60[i][j])            


### full data with different offertimes

In [13]:
pred_offer_times_list = [20,30,40,50,60]
pred_full_instance  = [choice_collection_20,choice_collection_30,choice_collection_40,choice_collection_50,choice_collection_60]
pred_full_lb = [lb_20,lb_30,lb_40,lb_50,lb_60]
pred_full_ub = [ub_20,ub_30,ub_40,ub_50,ub_60]

pred_full_collection = [collection_20,collection_30,collection_40,collection_50,collection_60]
pred_full_collection_size = [len(offer_times_20),len(offer_times_30),len(offer_times_40),len(offer_times_50),len(offer_times_60)]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
#pred_instance_size = [100,100,100,50,50]
pred_instance_size = [50,50,50,50,50]

price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

pred_full_offertimes = [offer_times_20,offer_times_30,offer_times_40,offer_times_50,offer_times_60]

train-test split

In [14]:
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(pred_full_collection[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(pred_full_collection[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(pred_full_collection[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = pred_full_instance[i][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = pred_full_lb[i][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = pred_full_ub[i][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(pred_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(pred_full_collection[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = pred_full_instance[i][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [15]:
pd.DataFrame(all_train_instances[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.735746,0.705128,0.647154,0.773333,0.738497,0.741758,0.736842,0.745098,0.847826,0.846774,...,0.75,0.692308,0.715447,0.791667,0.78125,0.822222,0.739130,0.642857,0.714286,0.688805
1,0.264254,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.176471,0.130435,0.048387,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.294872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.352846,0.000000,0.000000,0.000000,0.192982,0.000000,0.000000,0.000000,...,0.25,0.269231,0.211382,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.226667,0.000000,0.000000,0.000000,0.078431,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.18750,0.022222,0.043478,0.035714,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,...,0.00,0.038462,0.000000,0.000000,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.261503,0.000000,0.000000,0.000000,0.000000,0.104839,...,0.00,0.000000,0.073171,0.000000,0.00000,0.155556,0.000000,0.000000,0.142857,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.258242,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.217391,0.000000,0.142857,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.041667,0.00000,0.000000,0.000000,0.285714,0.000000,0.311195


In [22]:
pred_full_instance[0].shape

(9, 29)

In [16]:
pd.DataFrame(all_test_instance[0][0])

,0,1,2,3,4
0,0.91442,0.625,0.686567,0.748744,0.757576
1,0.00000,0.000,0.000000,0.000000,0.000000
2,0.00000,0.000,0.000000,0.000000,0.000000
3,0.00000,0.275,0.000000,0.000000,0.000000
4,0.00000,0.000,0.000000,0.000000,0.000000
5,0.08558,0.000,0.044776,0.060302,0.000000
6,0.00000,0.000,0.268657,0.000000,0.121212
7,0.00000,0.100,0.000000,0.000000,0.000000
8,0.00000,0.000,0.000000,0.190955,0.121212


In [17]:
pd.DataFrame(all_train_lb[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.690626,0.545579,0.605049,0.712343,0.705724,0.683877,0.556603,0.556516,0.684169,0.746813,...,0.538426,0.412595,0.589726,0.535492,0.555418,0.646098,0.456187,0.363030,0.478314,0.626481
1,0.219134,0.000000,0.000000,0.000000,0.000000,0.000000,-0.034380,0.011509,-0.023013,-0.011162,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.135323,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.310740,0.000000,0.000000,0.000000,0.031452,0.000000,0.000000,0.000000,...,0.038426,0.000414,0.097618,-0.068415,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.165676,0.000000,0.000000,0.000000,-0.037905,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.025720,-0.045682,-0.087926,-0.072662,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.044706,0.000000,...,0.000000,-0.078085,0.000000,0.000000,-0.063799,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.228729,0.000000,0.000000,0.000000,0.000000,0.019824,...,0.000000,0.000000,0.000609,0.000000,0.000000,-0.011405,0.000000,0.000000,-0.039925,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.200360,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.048388,0.000000,-0.039925,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.084382,0.000000,0.000000,0.000000,0.021890,0.000000,0.248872


In [18]:
pd.DataFrame(all_train_ub[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.780866,0.864677,0.689260,0.834324,0.771271,0.799640,0.917081,0.933680,1.011484,0.946735,...,0.961574,0.972020,0.841168,1.047841,1.007082,0.998346,1.022074,0.922685,0.950257,0.751128
1,0.309374,0.000000,0.000000,0.000000,0.000000,0.000000,0.174731,0.341432,0.283882,0.107936,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.454421,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.394951,0.000000,0.000000,0.000000,0.354513,0.000000,0.000000,0.000000,...,0.461574,0.538048,0.325146,0.401749,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.287657,0.000000,0.000000,0.000000,0.194768,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.400720,0.090127,0.174883,0.144091,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.088184,0.000000,...,0.000000,0.155009,0.000000,0.000000,0.126299,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.294276,0.000000,0.000000,0.000000,0.000000,0.189853,...,0.000000,0.000000,0.145732,0.000000,0.000000,0.322516,0.000000,0.000000,0.325640,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.316123,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.483170,0.000000,0.325640,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.167715,0.000000,0.000000,0.000000,0.549538,0.000000,0.373519


In [19]:
all_test_collection[0][0]

[[0, 5], [0, 3, 7], [0, 5, 6], [0, 5, 8], [0, 6, 8]]